In [3]:
from __future__ import print_function
import os
import time
import logging
import argparse
import numpy as np
from visdom import Visdom
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from utils import *
from metric.loss import AttentionTransfer

# Teacher models:
# VGG11/VGG13/VGG16/VGG19, GoogLeNet, AlxNet, ResNet18, ResNet34, 
# ResNet50, ResNet101, ResNet152, ResNeXt29_2x64d, ResNeXt29_4x64d, 
# ResNeXt29_8x64d, ResNeXt29_32x64d, PreActResNet18, PreActResNet34, 
# PreActResNet50, PreActResNet101, PreActResNet152, 
# DenseNet121, DenseNet161, DenseNet169, DenseNet201, 
import models

# Student models:
# myNet, LeNet, FitNet

start_time = time.time()

# Training settings
parser = argparse.ArgumentParser(description='PyTorch AT_KD CIFAR')

parser.add_argument('--dataset',
                    choices=['CIFAR10',
                             'CIFAR100'
                            ],
                    default='CIFAR10')
parser.add_argument('--teacher',
                    choices=['ResNet32',
                             'ResNet50',
                             'ResNet56',
                             'ResNet110'
                            ],
                    default='ResNet110')
parser.add_argument('--student',
                    choices=['ResNet20',
                             'myNet'
                            ],
                    default='ResNet20')
parser.add_argument('--at_ratio', default=1, type=float)
parser.add_argument('--kd_ratio', default=0.7, type=float)
parser.add_argument('--n_class', type=int, default=10, metavar='N', help='num of classes')
parser.add_argument('--T', type=float, default=20.0, metavar='Temputure', help='Temputure for distillation')
parser.add_argument('--batch_size', type=int, default=128, metavar='N', help='input batch size for training')
parser.add_argument('--test_batch_size', type=int, default=128, metavar='N', help='input test batch size for training')
parser.add_argument('--epochs', type=int, default=20, metavar='N', help='number of epochs to train (default: 20)')
parser.add_argument('--lr', type=float, default=0.1, metavar='LR', help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.9, metavar='M', help='SGD momentum (default: 0.5)')
parser.add_argument('--device', default='cuda:1', type=str, help='device: cuda or cpu')
parser.add_argument('--print_freq', type=int, default=10, metavar='N', help='how many batches to wait before logging training status')

config = ['--epochs', '200', '--at_ratio', '1', '--T', '5.0', '--device', 'cuda:1']
args = parser.parse_args(config)

device = args.device if torch.cuda.is_available() else 'cpu'
load_dir = './checkpoint/' + args.dataset + '/'

# teacher model
te_model = getattr(models, args.teacher)(num_classes=args.n_class)
te_model.load_state_dict(torch.load(load_dir + te_model.model_name + '.pth'))
te_model.to(device)

st_model = getattr(models, args.student)(num_classes=args.n_class)  # args.student()
st_model.to(device)

# logging
logfile = load_dir + 'AT_KD_' + st_model.model_name + '.log'
if os.path.exists(logfile):
    os.remove(logfile)
def log_out(info):
    f = open(logfile, mode='a')
    f.write(info)
    f.write('\n')
    f.close()
    print(info)
    
# visualizer
vis = Visdom(env='distill')
loss_win = vis.line(
    X=np.array([0]),
    Y=np.array([0]),
    opts=dict(
        title='AT+KD Loss',
        xtickmin=0,
#         xtickmax=1,
        xtickstep=5,
        ytickmin=0,
#         ytickmax=1,
        ytickstep=0.5,
#         markers=True,
#         markersymbol='dot',
#         markersize=5,
    ),
    name="loss"
)

acc_win = vis.line(
    X=np.column_stack((0, 0)),
    Y=np.column_stack((0, 0)),
    opts=dict(
        title='AT+KD Acc',
        xtickmin=0,
        xtickstep=5,
        ytickmin=0,
        ytickmax=100,
#         markers=True,
#         markersymbol='dot',
#         markersize=5,
        legend=['train_acc', 'test_acc']
    ),
    name="acc"
)


# data
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, 4),
    transforms.ToTensor(),
    normalize,
])
test_transform = transforms.Compose([transforms.ToTensor(), normalize])
train_set = getattr(datasets, args.dataset)(root='../data', train=True, download=True, transform=train_transform)
test_set = getattr(datasets, args.dataset)(root='../data', train=False, download=False, transform=test_transform)
train_loader = DataLoader(train_set, batch_size=args.batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=args.test_batch_size, shuffle=False)
# optim
optimizer_sgd = optim.SGD(st_model.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-4)
lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer_sgd, gamma=0.1, milestones=[100, 150])


# attention transfer loss
at_criterion = AttentionTransfer().to(device)

# avg diatill
def distillation_loss(y, labels, mean_logits, T, alpha=0.7):
    return nn.KLDivLoss()(F.log_softmax(y/T), mean_logits) * (T*T * 2.0 * alpha) + F.cross_entropy(y, labels) * (1. - alpha)


# train with multi-teacher
def train(epoch, st_model, te_model):
    print('Training:')
    # switch to train mode
    st_model.train()
    te_model.eval()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    
    end = time.time()
    for i, (input, target) in enumerate(train_loader):

        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.to(device), target.to(device)
        
        # compute outputs
        b1, b2, b3, pool, output = st_model(input)
        st_maps = [b1, b2, b3, pool]
        with torch.no_grad():
            t_b1, t_b2, t_b3, t_pool, t_output = te_model(input)
       
        hint_maps = [t_b1, t_b2, t_b3, t_pool]
        t_output = F.softmax(t_output/args.T)
        
        optimizer_sgd.zero_grad()
        
        # attention loss for each block/group
        at_loss = 0
        for idx in range(len(st_maps)):
            at_loss += at_criterion(st_maps[idx], hint_maps[idx])
        at_loss *= args.at_ratio

        kd_loss = distillation_loss(output, target, t_output, T=args.T, alpha=args.kd_ratio)
        loss = at_loss + kd_loss

        loss.backward(retain_graph=True)
        optimizer_sgd.step()

        output = output.float()
        loss = loss.float()
        # measure accuracy and record loss
        train_acc = accuracy(output.data, target.data)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(train_acc, input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            log_out('[{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                      i, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses, top1=top1))
    return losses.avg, train_acc.cpu().numpy()


def test(model):
    print('Testing:')
    # switch to evaluate mode
    model.eval()
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(test_loader):
            input, target = input.to(device), target.to(device)

            # compute output
            _,_,_,_,output = model(input)
            loss = F.cross_entropy(output, target)

            output = output.float()
            loss = loss.float()

            # measure accuracy and record loss
            test_acc = accuracy(output.data, target.data)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(test_acc, input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % args.print_freq == 0:
                log_out('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                          i, len(test_loader), batch_time=batch_time, loss=losses,
                          top1=top1))

    log_out(' * Prec@1 {top1.avg:.3f}'.format(top1=top1))

    return losses.avg, test_acc.cpu().numpy(), top1.avg.cpu().numpy()

# """
print('StudentNet:\n')
print(st_model)
# st_model.apply(weights_init_normal)
best_acc = 0
for epoch in range(1, args.epochs + 1):
    log_out("\n===> epoch: {}/{}".format(epoch, args.epochs))
    log_out('current lr {:.5e}'.format(optimizer_sgd.param_groups[0]['lr']))
    lr_scheduler.step(epoch)
    train_loss, train_acc = train(epoch, st_model, te_model)
    # visaulize loss
    vis.line(np.array([train_loss]), np.array([epoch]), loss_win, update="append")
    _, test_acc, top1 = test(st_model)
    vis.line(np.column_stack((train_acc, top1)), np.column_stack((epoch, epoch)), acc_win, update="append")
    if top1 > best_acc:
        best_acc = top1
        
# release GPU memory
torch.cuda.empty_cache()
log_out("BEST ACC: {:.3f}".format(best_acc))
log_out("--- {:.3f} mins ---".format((time.time() - start_time)/60))
# """

/home/data/yaliu/jupyterbooks/multi-KD/models/teacher/resnet20.py:36: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(m.weight)


Files already downloaded and verified
StudentNet:

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2

/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:179: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:150: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/torch/nn/functional.py:1992: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."


[0/391]	Time 0.481 (0.481)	Data 0.040 (0.040)	Loss 3.0635 (3.0635)	Prec@1 14.844 (14.844)
[10/391]	Time 0.051 (0.092)	Data 0.018 (0.021)	Loss 2.4729 (2.7715)	Prec@1 15.625 (16.406)
[20/391]	Time 0.051 (0.073)	Data 0.018 (0.020)	Loss 2.3510 (2.5823)	Prec@1 15.625 (20.350)
[30/391]	Time 0.051 (0.066)	Data 0.018 (0.019)	Loss 2.3841 (2.5015)	Prec@1 24.219 (21.749)
[40/391]	Time 0.051 (0.062)	Data 0.019 (0.019)	Loss 2.2762 (2.4421)	Prec@1 32.812 (22.847)
[50/391]	Time 0.051 (0.060)	Data 0.018 (0.019)	Loss 2.2635 (2.4029)	Prec@1 23.438 (23.269)
[60/391]	Time 0.058 (0.059)	Data 0.026 (0.019)	Loss 2.2668 (2.3649)	Prec@1 33.594 (24.411)
[70/391]	Time 0.051 (0.058)	Data 0.018 (0.019)	Loss 1.9247 (2.3293)	Prec@1 30.469 (25.627)
[80/391]	Time 0.051 (0.057)	Data 0.018 (0.019)	Loss 1.9431 (2.3011)	Prec@1 42.969 (26.591)
[90/391]	Time 0.051 (0.056)	Data 0.018 (0.019)	Loss 2.1706 (2.2746)	Prec@1 32.031 (27.515)
[100/391]	Time 0.051 (0.056)	Data 0.018 (0.019)	Loss 2.1289 (2.2534)	Prec@1 39.062 (28.110)

Test: [30/79]	Time 0.019 (0.018)	Loss 1.9621 (1.8937)	Prec@1 52.344 (52.016)
Test: [40/79]	Time 0.018 (0.018)	Loss 1.7556 (1.8693)	Prec@1 52.344 (52.611)
Test: [50/79]	Time 0.018 (0.018)	Loss 1.9418 (1.8823)	Prec@1 57.031 (52.543)
Test: [60/79]	Time 0.018 (0.018)	Loss 1.8636 (1.8807)	Prec@1 54.688 (52.754)
Test: [70/79]	Time 0.018 (0.018)	Loss 1.5278 (1.8843)	Prec@1 60.156 (52.674)
 * Prec@1 52.790

===> epoch: 3/200
current lr 1.00000e-01
Training:
[0/391]	Time 0.052 (0.052)	Data 0.019 (0.019)	Loss 1.1249 (1.1249)	Prec@1 69.531 (69.531)
[10/391]	Time 0.051 (0.052)	Data 0.019 (0.019)	Loss 1.2334 (1.2088)	Prec@1 60.938 (64.773)
[20/391]	Time 0.051 (0.051)	Data 0.018 (0.019)	Loss 1.2576 (1.2180)	Prec@1 60.938 (64.583)
[30/391]	Time 0.051 (0.051)	Data 0.018 (0.019)	Loss 1.1541 (1.2151)	Prec@1 68.750 (64.970)
[40/391]	Time 0.051 (0.051)	Data 0.018 (0.019)	Loss 1.2381 (1.2049)	Prec@1 63.281 (65.320)
[50/391]	Time 0.051 (0.051)	Data 0.018 (0.018)	Loss 1.3599 (1.2017)	Prec@1 67.188 (65.349)
[

...

[70/391]	Time 0.052 (0.054)	Data 0.018 (0.018)	Loss 0.0874 (0.0960)	Prec@1 99.219 (98.779)
[80/391]	Time 0.052 (0.053)	Data 0.018 (0.018)	Loss 0.1392 (0.0957)	Prec@1 98.438 (98.804)
[90/391]	Time 0.053 (0.053)	Data 0.018 (0.018)	Loss 0.0954 (0.0953)	Prec@1 99.219 (98.824)
[100/391]	Time 0.052 (0.053)	Data 0.018 (0.018)	Loss 0.0874 (0.0946)	Prec@1 98.438 (98.855)
[110/391]	Time 0.054 (0.053)	Data 0.018 (0.018)	Loss 0.1144 (0.0945)	Prec@1 97.656 (98.839)
[120/391]	Time 0.053 (0.053)	Data 0.018 (0.018)	Loss 0.1211 (0.0950)	Prec@1 96.875 (98.799)
[130/391]	Time 0.053 (0.053)	Data 0.018 (0.018)	Loss 0.1235 (0.0952)	Prec@1 99.219 (98.807)
[140/391]	Time 0.053 (0.053)	Data 0.018 (0.018)	Loss 0.0849 (0.0954)	Prec@1 98.438 (98.742)
[150/391]	Time 0.053 (0.053)	Data 0.019 (0.018)	Loss 0.0985 (0.0948)	Prec@1 98.438 (98.769)
[160/391]	Time 0.053 (0.053)	Data 0.018 (0.018)	Loss 0.0833 (0.0942)	Prec@1 99.219 (98.787)
[170/391]	Time 0.054 (0.053)	Data 0.018 (0.018)	Loss 0.0690 (0.0937)	Prec@1 100.000

[10/391]	Time 0.056 (0.057)	Data 0.018 (0.019)	Loss 0.0773 (0.0988)	Prec@1 100.000 (98.722)
[20/391]	Time 0.056 (0.056)	Data 0.018 (0.019)	Loss 0.0809 (0.0923)	Prec@1 99.219 (98.735)
[30/391]	Time 0.054 (0.056)	Data 0.018 (0.019)	Loss 0.1026 (0.0929)	Prec@1 98.438 (98.866)
[40/391]	Time 0.054 (0.055)	Data 0.018 (0.019)	Loss 0.0706 (0.0900)	Prec@1 99.219 (98.952)
[50/391]	Time 0.053 (0.055)	Data 0.018 (0.019)	Loss 0.0797 (0.0906)	Prec@1 98.438 (98.974)
[60/391]	Time 0.053 (0.055)	Data 0.018 (0.018)	Loss 0.0995 (0.0912)	Prec@1 98.438 (98.988)
[70/391]	Time 0.052 (0.054)	Data 0.018 (0.018)	Loss 0.1278 (0.0941)	Prec@1 98.438 (98.944)
[80/391]	Time 0.052 (0.054)	Data 0.018 (0.018)	Loss 0.1010 (0.0934)	Prec@1 98.438 (98.939)
[90/391]	Time 0.053 (0.054)	Data 0.018 (0.018)	Loss 0.0700 (0.0928)	Prec@1 100.000 (98.944)
[100/391]	Time 0.052 (0.054)	Data 0.018 (0.018)	Loss 0.0820 (0.0924)	Prec@1 99.219 (98.948)
[110/391]	Time 0.054 (0.054)	Data 0.018 (0.018)	Loss 0.0879 (0.0923)	Prec@1 99.219 (98.

Test: [30/79]	Time 0.018 (0.018)	Loss 0.3896 (0.3625)	Prec@1 93.750 (92.591)
Test: [40/79]	Time 0.018 (0.018)	Loss 0.3737 (0.3598)	Prec@1 90.625 (92.530)
Test: [50/79]	Time 0.018 (0.018)	Loss 0.3974 (0.3554)	Prec@1 92.969 (92.586)
Test: [60/79]	Time 0.018 (0.018)	Loss 0.2083 (0.3540)	Prec@1 96.094 (92.623)
Test: [70/79]	Time 0.019 (0.018)	Loss 0.4570 (0.3530)	Prec@1 91.406 (92.584)
 * Prec@1 92.630

===> epoch: 196/200
current lr 1.00000e-03
Training:
[0/391]	Time 0.056 (0.056)	Data 0.020 (0.020)	Loss 0.0914 (0.0914)	Prec@1 98.438 (98.438)
[10/391]	Time 0.055 (0.055)	Data 0.018 (0.018)	Loss 0.1086 (0.0988)	Prec@1 100.000 (98.722)
[20/391]	Time 0.055 (0.055)	Data 0.018 (0.018)	Loss 0.1143 (0.0935)	Prec@1 96.875 (98.772)
[30/391]	Time 0.055 (0.055)	Data 0.018 (0.018)	Loss 0.0710 (0.0922)	Prec@1 100.000 (98.841)
[40/391]	Time 0.054 (0.055)	Data 0.018 (0.018)	Loss 0.0796 (0.0921)	Prec@1 99.219 (98.857)
[50/391]	Time 0.052 (0.055)	Data 0.018 (0.018)	Loss 0.0904 (0.0905)	Prec@1 99.219 (98.89

[370/391]	Time 0.051 (0.053)	Data 0.018 (0.018)	Loss 0.0775 (0.0906)	Prec@1 100.000 (98.930)
[380/391]	Time 0.051 (0.053)	Data 0.018 (0.018)	Loss 0.0806 (0.0909)	Prec@1 99.219 (98.919)
[390/391]	Time 0.035 (0.053)	Data 0.012 (0.018)	Loss 0.0923 (0.0908)	Prec@1 98.750 (98.928)
Testing:
Test: [0/79]	Time 0.019 (0.019)	Loss 0.4059 (0.4059)	Prec@1 94.531 (94.531)
Test: [10/79]	Time 0.018 (0.018)	Loss 0.3725 (0.3523)	Prec@1 92.969 (92.827)
Test: [20/79]	Time 0.018 (0.018)	Loss 0.2071 (0.3628)	Prec@1 94.531 (92.597)
Test: [30/79]	Time 0.018 (0.018)	Loss 0.3552 (0.3636)	Prec@1 94.531 (92.591)
Test: [40/79]	Time 0.018 (0.018)	Loss 0.4248 (0.3628)	Prec@1 89.062 (92.530)
Test: [50/79]	Time 0.018 (0.018)	Loss 0.3669 (0.3539)	Prec@1 92.969 (92.601)
Test: [60/79]	Time 0.018 (0.018)	Loss 0.1896 (0.3492)	Prec@1 95.312 (92.738)
Test: [70/79]	Time 0.019 (0.018)	Loss 0.4559 (0.3464)	Prec@1 91.406 (92.738)
 * Prec@1 92.770

===> epoch: 198/200
current lr 1.00000e-03
Training:
[0/391]	Time 0.057 (0.057)	D

[310/391]	Time 0.054 (0.054)	Data 0.018 (0.018)	Loss 0.0832 (0.0907)	Prec@1 99.219 (98.917)
[320/391]	Time 0.055 (0.054)	Data 0.018 (0.018)	Loss 0.0923 (0.0909)	Prec@1 100.000 (98.917)
[330/391]	Time 0.055 (0.054)	Data 0.018 (0.018)	Loss 0.1169 (0.0910)	Prec@1 99.219 (98.910)
[340/391]	Time 0.054 (0.054)	Data 0.018 (0.018)	Loss 0.0755 (0.0912)	Prec@1 100.000 (98.912)
[350/391]	Time 0.052 (0.054)	Data 0.018 (0.018)	Loss 0.0683 (0.0911)	Prec@1 99.219 (98.900)
[360/391]	Time 0.052 (0.054)	Data 0.018 (0.018)	Loss 0.0747 (0.0911)	Prec@1 100.000 (98.898)
[370/391]	Time 0.051 (0.054)	Data 0.018 (0.018)	Loss 0.1213 (0.0916)	Prec@1 96.875 (98.882)
[380/391]	Time 0.053 (0.054)	Data 0.018 (0.018)	Loss 0.1093 (0.0918)	Prec@1 97.656 (98.876)
[390/391]	Time 0.035 (0.053)	Data 0.012 (0.018)	Loss 0.0713 (0.0915)	Prec@1 100.000 (98.892)
Testing:
Test: [0/79]	Time 0.018 (0.018)	Loss 0.3766 (0.3766)	Prec@1 92.969 (92.969)
Test: [10/79]	Time 0.018 (0.018)	Loss 0.3359 (0.3457)	Prec@1 93.750 (92.898)
Test: 